In [3]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error



In [4]:
#Gets the data from the scraper and deletes the index column
stats = pd.read_csv('total.csv')
del stats['Unnamed: 0']
raptor = pd.read_csv('modern_RAPTOR_by_player.csv')

In [5]:
raptor_clean = raptor[['player_name','raptor_total', 'season', 'mp']]

columns_to_rename = {'player_name': 'Player','raptor_total': 'Raptor'}
raptor_clean = raptor_clean.rename(columns = columns_to_rename)
raptor_clean['Raptor'] = round(raptor_clean['Raptor'], 1)

In [6]:
#Missing data

#2000-2012 (Shuttlerun)
#2014, 16, 21, 22, 23 (Bench Press)

#Get total mp
#Remove duplicate names

shuttleRun = range(2000,2013)
benchPress = [2014,2016,2021,2022,2023]
stats

SEASON  PLAYER_ID FIRST_NAME   LAST_NAME             Player POSITION  \
0      2003      12080     Justin    Hamilton    Justin Hamilton    PG-SG   
1      2009     201977     Marcus    Thornton    Marcus Thornton       SG   
2      2009     201977     Marcus    Thornton    Marcus Thornton       SG   
3      2015    1626194     Marcus    Thornton    Marcus Thornton    PG-SG   
4      2015    1626194     Marcus    Thornton    Marcus Thornton    PG-SG   
..      ...        ...        ...         ...                ...      ...   
320    2021    1630531      Jaden    Springer     Jaden Springer       SG   
321    2021    1630580        Joe    Wieskamp       Joe Wieskamp       SF   
322    2021    1630598      Aaron     Wiggins      Aaron Wiggins    SG-SF   
323    2021    1630533     Ziaire    Williams    Ziaire Williams       SF   
324    2021    1630681     Marcus  Zegarowski  Marcus Zegarowski       PG   

     HEIGHT_WO_SHOES  WEIGHT  WINGSPAN  STANDING_REACH  ...  TRB  AST  STL  \
0              75.00   217.0     83.00           103.0  ...  7.2  0.9  0.8   
1              74.75   194.4     77.00            99.0  ...  7.3  1.0  0.7   
2              74.75   194.4     77.00            99.0  ...  2.8  2.9  0.6   
3              73.75   177.4     79.75            97.5  ...  7.3  1.0  0.7   
4              73.75   177.4     79.75            97.5  ...  2.8  2.9  0.6   
..               ...     ...       ...             ...  ...  ...  ...  ...   
320            75.00   202.0     79.75            99.0  ...  3.5  2.9  1.2   
321            77.75   204.8     83.00           103.0  ...  6.6  1.7  0.9   
322            76.50   190.0     81.75           103.0  ...  5.8  2.5  1.1   
323            80.25   188.4     82.25           106.5  ...  4.6  2.2  0.9   
324            72.75   180.8     74.75            96.0  ...  3.6  4.3  1.3   

     BLK   PER   WS  BPM  Year  Pick  Age  
0    1.3  22.7  4.3  5.9  2012    45   22  
1    1.1  20.8  3.8  5.5  2015    45   22  
2    0.1  21.8  4.7  1.3  2015    45   22  
3    1.1  20.8  3.8  5.5  2015    45   22  
4    0.1  21.8  4.7  1.3  2015    45   22  
..   ...   ...  ...  ...   ...   ...  ...  
320  0.4  20.9  2.9  6.9  2021    28   18  
321  0.3  22.7  4.1  9.3  2021    41   21  
322  0.5  19.9  3.7  7.0  2021    55   22  
323  0.6  11.2  0.6  0.3  2021    10   19  
324  0.1  20.9  4.3  8.1  2021    49   22  

[325 rows x 33 columns]

In [6]:
#Gets Predidctor variables
def get_predictors(year):
        columns_to_drop = ['SEASON',
                         'PLAYER_ID',
                         'FIRST_NAME',
                         'LAST_NAME',
                         'Player',
                         'POSITION',
                         'PER',
                         'BPM',
                         'Year',
                         'Pick',
                         'Age',
                         'Pos',
                         'STANDING_REACH_FT_IN']

        predictors = (stats.drop(columns = columns_to_drop)).columns
        
        if year in shuttleRun:
            predictors.drop(columns = year)
        elif year in benchPress:
            

        
    


In [7]:
def choose_year(year):
    
    #Training the model 
    train = stats[stats["Year"] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha = 0.001)
    reg.fit(train[predictors], train['Pick'])

    #Getting predicitons in a dataframe
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns = ['Predictions'], index = test.index)
    combination = pd.concat([test[['Player', 'Pick']], predictions], axis = 1)
    
    #Getting Mean Square Error
    mse = mean_squared_error(combination['Pick'], combination['Predictions'])
    string = 'The mean square error for this draft is: ' + str(mse)
    
    #Displays rank which refers to each players draft position relative to one another in the given dataset as well as the predicted rank and difference
    #Not used cause clutter
    combination = combination.sort_values('Pick', ascending = True)
    combination['RK'] = list(range(1, combination.shape[0]+1))
    combination = combination.sort_values('Predictions', ascending = True)
    combination['Predicted_RK'] = list(range(1, combination.shape[0]+1))
    combination['Difference'] = combination['Predicted_RK'] - combination['RK']

    #Gets a clutter free df 'clean' which neatly displays information
    important_columns = ['Player',  'Pick',  'Predictions']
    clean = combination.loc[:,important_columns]
    clean['Predictions'] = round(clean['Predictions'], 0)
    clean['Predictions'] = clean['Predictions'].astype(int)
    clean['Difference'] = combination['Pick'] - clean['Predictions'] 
    
    #Raptor, next step is use outer merge to get players with 0 min play
    final = pd.merge(clean, raptor_clean[raptor_clean['season'] == 2021], on='Player', how = 'left')
    final = final.fillna(0)
    final['Score'] = (final['Pick'] -final['Difference'].mean() - final['Predictions']) *  final['mp']
    score = sum(final['Score'])
    return final, score

    

In [8]:
#year = input("Enter the draft class you would like to predict: ")


In [9]:
#choose_year(int(year))

In [10]:
choose_year(int(2015))[0]

Player  Pick  Predictions  Difference  Raptor  season  \
0   Rondae Hollis-Jefferson    23            8          15    -3.0  2021.0   
1                Sam Dekker    18           11           7     0.0     0.0   
2             Norman Powell    46           19          27     0.2  2021.0   
3           Justin Anderson    21           20           1     0.0     0.0   
4               R.J. Hunter    28           22           6     0.0     0.0   
5             Cameron Payne    14           24         -10     4.2  2021.0   
6           Marcus Thornton    45           25          20     0.0     0.0   
7              Terry Rozier    16           25          -9     1.6  2021.0   
8               J.P. Tokoto    58           26          32     0.0     0.0   
9              Delon Wright    20           26          -6     2.6  2021.0   
10           Branden Dawson    56           26          30     0.0     0.0   
11             Devin Booker    13           26         -13     2.7  2021.0   
12            Jarell Martin    25           27          -2     0.0     0.0   
13          Marcus Thornton    45           29          16     0.0     0.0   
14           Richaun Holmes    37           29           8     2.1  2021.0   
15           Dakari Johnson    48           31          17     0.0     0.0   
16            Anthony Brown    34           31           3     0.0     0.0   
17             Bobby Portis    22           32         -10     2.7  2021.0   
18          Andrew Harrison    44           33          11     0.0     0.0   
19         Rakeem Christmas    36           33           3     0.0     0.0   
20          Pat Connaughton    41           34           7     1.8  2021.0   
21           Olivier Hanlan    42           38           4     0.0     0.0   
22            Rashad Vaughn    17           38         -21     0.0     0.0   
23            Jordan Mickey    33           38          -5     0.0     0.0   
24               Tyus Jones    24           47         -23    -0.2  2021.0   
25          Marcus Thornton    45           47          -2     0.0     0.0   
26             Tyler Harvey    51           48           3     0.0     0.0   
27          Marcus Thornton    45           51          -6     0.0     0.0   

        mp         Score  
0    143.0   1618.964286  
1      0.0      0.000000  
2   2421.0  56461.178571  
3      0.0     -0.000000  
4      0.0      0.000000  
5   1497.0 -20476.821429  
6      0.0      0.000000  
7   2383.0 -30213.035714  
8      0.0      0.000000  
9   1748.0 -16918.142857  
10     0.0      0.000000  
11  3159.0 -52687.607143  
12     0.0     -0.000000  
13     0.0      0.000000  
14  1782.0   7700.785714  
15     0.0      0.000000  
16     0.0     -0.000000  
17  1738.0 -23773.357143  
18     0.0      0.000000  
19     0.0     -0.000000  
20  2121.0   7044.750000  
21     0.0      0.000000  
22     0.0     -0.000000  
23     0.0     -0.000000  
24  1269.0 -33855.107143  
25     0.0     -0.000000  
26     0.0     -0.000000  
27     0.0     -0.000000

In [11]:
test = 0
for year in range(2014,2021):
    print(year)
    print(choose_year(int(year))[1])
    test = test + choose_year(int(year))[1]
test

2014
-179174.38709677415
2015
-105098.39285714287
2016
-38748.25
2017
-65277.57575757573
2018
-96952.0
2019
-54204.181818181816
2020
-30987.999999999996


-570442.7875296746

In [63]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import draftcombinestats
from nba_api.stats.endpoints import commonplayerinfo
def get_info(name):
    player_dict = players.get_players()
    for player in player_dict:
        if player['full_name'] == name:
            player_info = commonplayerinfo.CommonPlayerInfo(player_id=player['id'])
            df = player_info.get_data_frames()
            df[1]['SCHOOL'] = df[0]['SCHOOL']
    
    
    return(df[1])
    
    

In [64]:
a = get_info('Steven Adams')
a

PLAYER_ID   PLAYER_NAME TimeFrame  PTS  AST   REB    PIE      SCHOOL
0     203500  Steven Adams   2021-22  6.9  3.4  10.0  0.108  Pittsburgh

{'id': 76001,
 'full_name': 'Alaa Abdelnaby',
 'first_name': 'Alaa',
 'last_name': 'Abdelnaby',
 'is_active': False}